In [1]:
import pandas as pd
df = pd.read_csv('../data/Original_csv/Hyperion.csv', na_filter=False)

In [2]:
import re
#df.drop('Domanda', inplace=True, axis=1)
#df.drop('Repertorio', inplace=True, axis=1)

#filter = df["Repertorio"] != ""
#df = df[filter]

# lower entire dataset
#df = df.applymap(str.lower)

#df['Repertorio'].replace('implicazioni','implicazione', inplace=True)
#df['Repertorio'].replace('previsioni','previsione', inplace=True)



In [3]:
import string

dataset = []
sample = {}

for row in df.itertuples():
    if row.Testo:
        dataset.append(sample)
        sample = {}
        sample['Testo'] = row.Testo.replace("\n", " ")
        sample['Stralci'] = [row.Stralcio.replace("\n", " ")]
        sample['Repertori'] = [row.Repertorio]
        
    if not row.Testo:
        sample['Stralci'].append(row.Stralcio.replace("\n", " "))
        sample['Repertori'].append(row.Repertorio)
del dataset[0]

#Find bounds starting froma text
def find_char_bounds(spans: list, text: str) -> list:
    bounds = []
    last_char = 0
    for span in spans:
        start = text.find(span)
        if start == -1:
            start = last_char + 1
        bounds.append((start, start + len(span)))
        last_char = start + len(span)
    return bounds

def find_word_bounds(spans: list, text: str) -> list:
    bounds = []
    start = 0
    for span in spans:
        s = span.translate(str.maketrans('', '', string.punctuation))
        word_list = s.split()
        text_list = text.translate(str.maketrans('', '', string.punctuation)).split()
        try:
            start = text_list.index(word_list[0], start)
        except:
            if not bounds:
                start = 0
            else:
                start = bounds[-1][1] +1
            
        bounds.append((start, start + len(word_list)))
    return bounds


for sample in dataset:
    #sample['Bounds'] = find_char_bounds(sample['Stralci'], sample['Testo'])
    sample['Bounds'] = find_word_bounds(sample['Stralci'], sample['Testo'])



In [4]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
import re

nltk_pred = []

for sample in dataset:
    tokens = sent_tokenize(sample['Testo'])
    spans = []
    bounds = []
    for x in tokens:
        #spans += re.findall('.*?[.:!?;,]', x)
        spans += re.split('[.:!?;,]', x)
        spans = list(filter(None, spans))




    #bounds += find_char_bounds(spans, sample['Testo'])
    bounds += find_word_bounds(spans, sample['Testo'])
    nltk_pred.append(bounds)

print(dataset[0]['Bounds'])
nltk_pred[0]


    



[nltk_data] Downloading package punkt to /home/michele/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[(0, 5), (5, 13), (13, 23), (23, 38), (38, 45), (45, 47)]


[(0, 5), (5, 13), (13, 23), (23, 38), (38, 45), (45, 47)]

In [5]:
def intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    #yA = max(boxA[1], boxB[1])
    xB = min(boxA[1], boxB[1])
    #yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[1] - boxA[0]))
    boxBArea = abs((boxB[1] - boxB[0]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [6]:
import numpy as np
# A è B sono tupe con i bound dello span
def IoU(A, B):
    if A == B:
        return 1
    start = max(A[0], B[0])
    end = min(A[1], B[1])
    if(start > end):
        return 0
    intersection = end - start
    return intersection / (A[1] - A[0] + B[1] - B[0] - intersection)

def compute_IoUs(pred_bounds, gt_spans):
    IoUs = []
    for gt_bounds in gt_spans:
        IoUs.append(IoU(pred_bounds, gt_bounds)) 
    return IoUs

#Input: text_spans_dict = [ {
#           'Bounds' : (a,b), 
#           'IoU' : float,
#           'Repertorio': 'string':
#           } ]
def normalize(text_spans_dict, gt_spans):
    normalized = []
    for i in range(len(text_spans_dict)):
        #normalized is not empty
        if normalized:
            if normalized[-1]['Repertorio'] == text_spans_dict[i]['Repertorio']:
                new_span = (normalized[-1]['Bounds'][0], text_spans_dict[i]['Bounds'][1])
                new_span_features = {
                    'Bounds' : new_span, 
                    'IoU' : None,
                    'Repertorio' : text_spans_dict[i]['Repertorio']
                    }
                del normalized[-1]
                normalized.append(new_span_features)
            else:
                normalized.append(text_spans_dict[i])
        else:
            normalized.append(text_spans_dict[i])
        
    
    for i in range(len(normalized)):
        normalized[i]['IoU'] = max(compute_IoUs(normalized[i]['Bounds'], gt_spans['Bounds']))
    return normalized
    

metrics = []
normalized_metrics = []
for i, pred_bounds in enumerate(nltk_pred):
    text_IoUs = []
    for pred_span in pred_bounds:
        IoUs = compute_IoUs(pred_span, dataset[i]['Bounds'])
        best = np.argmax(IoUs)
        span_features = {
            'Bounds' : pred_span, 
            'IoU' : IoUs[best],
            'Repertorio' : dataset[i]['Repertori'][best]
            }

        text_IoUs.append(span_features)
    metrics.append(text_IoUs)
    normalized_metrics.append(normalize(text_IoUs, dataset[i]))


        
#TODO non funziona normalizzazione


In [7]:
print(metrics[10])
print(normalized_metrics[10])
print(dataset[10])

[{'Bounds': (0, 8), 'IoU': 0.7272727272727273, 'Repertorio': 'Non risposta'}, {'Bounds': (8, 11), 'IoU': 0.2727272727272727, 'Repertorio': 'Non risposta'}, {'Bounds': (11, 14), 'IoU': 0.75, 'Repertorio': 'Commento'}, {'Bounds': (11, 12), 'IoU': 0.25, 'Repertorio': 'Commento'}, {'Bounds': (15, 23), 'IoU': 1, 'Repertorio': 'Giudizio'}]
[{'Bounds': (0, 11), 'IoU': 1, 'Repertorio': 'Non risposta'}, {'Bounds': (11, 12), 'IoU': 0.25, 'Repertorio': 'Commento'}, {'Bounds': (15, 23), 'IoU': 1, 'Repertorio': 'Giudizio'}]
{'Testo': 'Chi non vuole il MES scenda in piazza. Coronavirus o no. Basta social network. Basta. Siamo uomini e donne o larve da tastiera?', 'Stralci': ['Chi non vuole il MES scenda in piazza. Coronavirus o no. ', 'Basta social network. Basta', 'Siamo uomini e donne o larve da tastiera?'], 'Repertori': ['Non risposta', 'Commento', 'Giudizio'], 'Bounds': [(0, 11), (11, 15), (15, 23)]}


In [8]:
n_spans = 0
for e in dataset:
    n_spans += len(e['Bounds'])
print('Numero stralci nel dataset:', str(n_spans))

n_spans = 0
for e in normalized_metrics:
    n_spans += len(e)
print('Numero stralci predetti:', str(n_spans))

mean = 0
long_spans = 0
min_lenght = 0
perfect_spans =0
for text in normalized_metrics:
    for span in text:
        if span['Bounds'][1] - span['Bounds'][0] >= min_lenght:
            long_spans += 1
            mean += span['IoU']
            if span['IoU'] == 1:
                perfect_spans += 1
perfect_spans_perc = perfect_spans / long_spans
mean_IoU = mean / long_spans
print('Numero stralci con lunghezza minima = ', str(min_lenght), ': ', str(long_spans))
print('Media IoU:', str(mean_IoU))
print('Percentuale span perfetti: ', str(perfect_spans_perc))




Numero stralci nel dataset: 35471
Numero stralci predetti: 32550
Numero stralci con lunghezza minima =  0 :  32550
Media IoU: 0.8713763257225484
Percentuale span perfetti:  0.6668817204301075
